# Import the necessary libraries

In [0]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,GRU
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.models import LdaModel
from gensim import models, corpora, similarities
import re
from nltk.stem.porter import PorterStemmer
import time
from nltk import FreqDist
from scipy.stats import entropy
from subprocess import check_output
import io
import matplotlib
from zipfile import ZipFile
from tqdm import tqdm
from bs4 import BeautifulSoup
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
from sklearn.metrics import f1_score
from keras.callbacks import ModelCheckpoint
%matplotlib inline

matplotlib.style.use('ggplot')
sns.set_style("darkgrid")

In [69]:
# Load trained fasttext embedding 300 Vector dimension
 !wget 'https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip'
#Load news data from amazonaws for propaganda-datathon
 !wget 'https://s3.us-east-2.amazonaws.com/propaganda-datathon/dataset/datasets-v5.zip'


--2020-02-17 08:26:51--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘crawl-300d-2M.vec.zip.1’

crawl-300d-2M.vec.z 100%[===================>]   1.42G  11.8MB/s    in 2m 5s   

2020-02-17 08:28:57 (11.7 MB/s) - ‘crawl-300d-2M.vec.zip.1’ saved [1523785255/1523785255]

--2020-02-17 08:28:59--  https://s3.us-east-2.amazonaws.com/propaganda-datathon/dataset/datasets-v5.zip
Resolving s3.us-east-2.amazonaws.com (s3.us-east-2.amazonaws.com)... 52.219.84.186
Connecting to s3.us-east-2.amazonaws.com (s3.us-east-2.amazonaws.com)|52.219.84.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50012761 (48M) [application/zip]
Saving

In [0]:
max_words = 100000 
max_len = 500

# Load and prepare data from the folder

In [0]:
from gensim.models.keyedvectors import KeyedVectors
import os

class EmbedLoader:
    

    def __init__(self):
        pass

    def get_embedding(self, embedding_file, embed_size):
        embedding_index = {}
        with open(embedding_file, encoding='utf8') as f:
            for line in f:
                values = line.rstrip().rsplit(' ')
                # word = ' '.join(values[:-embed_size])
                # coefs = np.asarray(values[-embed_size:], dtype='float32')
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                embedding_index[word] = coefs
        f.close()

        return embedding_index

    def get_embedding2(self, embedding_file):
        embeddings_index = dict(self._get_coefs(*o.rstrip().split(' ')) for o in open(embedding_file))
        return embeddings_index

    def _get_coefs(self,word, *arr):
        return word, np.asarray(arr, dtype='float32')

    def get_embedding_matrix(self, embedding_index, word_index, embed_size, max_features):

        print("size word index: " , len(word_index))
        vocab_size = min(len(word_index), max_features) + 1
        embedding_matrix = np.zeros((vocab_size, embed_size))

        print("embedding_matrix shape: ", embedding_matrix.shape)

        for word, index in word_index.items():
            if index >= vocab_size:
                continue
            embedding_vector = embedding_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[index] = embedding_vector

        return embedding_matrix, vocab_size



Define a swish activation function

In [0]:
class Swish(Activation):
    
    def __init__(self, activation, **kwargs):
        super(Swish, self).__init__(activation, **kwargs)
        self.__name__ = 'swish'

def swish(x):
    return (K.sigmoid(x) * x)

get_custom_objects().update({'swish': Swish(swish)})

In [0]:
def prepare_data(data_path):
  # Create a ZipFile Object and load sample.zip in it
  with ZipFile(data_path+"/"+"datasets-v5.zip", "r") as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()
  f = io.open(data_path+"/"+"datasets-v5/task-1/task1.train.txt", mode="r", encoding="utf-8")
  # df = pd.DataFrame(columns=["text","article_id","propaganda"])
  lines = f.read().split("\n")
  x = []
  for i in lines:
    i = i.split("\t")
    x.append(i)
  df = pd.DataFrame(x, columns=["text","article_id","propaganda"])
  # Load nltk stopwords
  nltk.download('stopwords')
  # stopWords = set(stopwords.words('english'))
  # MY STOPWORDS
  stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't", "no", "nor","not"])
  preprocessed_text = []
  # tqdm is for printing the status bar
  for sentance in tqdm(df['text'].values):
      sentance = re.sub(r"http\S+", "", sentance)
      sentance = BeautifulSoup(sentance, 'lxml').get_text()
      #sentance = decontracted(sentance)
      sentance = re.sub("\S*\d\S*", "", sentance).strip()
      sentance = re.sub('[^A-Za-z]+', ' ', sentance)
      # https://gist.github.com/sebleier/554280
      sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
      preprocessed_text.append(sentance.strip())
  df["text"] = preprocessed_text
  df = df.dropna()
  df['length'] = [len(text) for text in df['text']]
  df = df[df.length < 40000]
  X = df.text
  Y = df.propaganda
  le = LabelEncoder()
  Y = le.fit_transform(Y)
  Y = Y.reshape(-1,1)
  X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)
  X_all =  pd.concat([X_train, X_test], ignore_index=True)
  tok = Tokenizer(num_words=max_words)
  tok.fit_on_texts(X_all)
  sequences = tok.texts_to_sequences(X_train)
  sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
  
  # extract and load the embedding file
  with ZipFile('crawl-300d-2M.vec.zip', 'r') as zip:
    zip.extract('crawl-300d-2M.vec')
  embed_loader = EmbedLoader()
  embedding_index = embed_loader.get_embedding2('crawl-300d-2M.vec')
  embedding_matrix, vocab_size = embed_loader.get_embedding_matrix(embedding_index, tok.word_index,300, max_words)
  return embedding_matrix, vocab_size,X_train,X_test,Y_train,Y_test


In [82]:
embedding_matrix ,vocab_size ,X_train,X_test,Y_train,Y_test =  prepare_data("/content")

  0%|          | 64/35994 [00:00<00:56, 639.07it/s]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


100%|██████████| 35994/35994 [00:44<00:00, 817.23it/s]


size word index:  162019
embedding_matrix shape:  (100001, 300)


### RNN
Define the RNN structure.

In [0]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    embed_size = 300
    emb_layer = Embedding(vocab_size, embed_size, weights=[embedding_matrix], trainable=False, name='embedding_1')(inputs)
    layer = emb_layer
    layer = LSTM(256,return_sequences = True)(layer)
    layer = LSTM(128)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('swish')(layer)
    layer = Dropout(0.25)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

Call the function and compile the model.

In [84]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 500)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 500, 300)          30000300  
_________________________________________________________________
lstm_3 (LSTM)                (None, 500, 256)          570368    
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               197120    
_________________________________________________________________
FC1 (Dense)                  (None, 256)               33024     
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0   

Fit on the training data.

In [0]:
early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=2,
                          verbose=2, mode='auto')
checkpointer = ModelCheckpoint(filepath='model_check.hdf5', verbose=1, monitor='val_loss', save_best_only=True)

In [0]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[early_stopping, checkpointer ])

Train on 24472 samples, validate on 6119 samples
Epoch 1/10
24448/24472 [============================>.] - ETA: 1s - loss: 0.3566 - acc: 0.8876 

In [0]:
# load best score
model.load_weights('model_check.hdf5')

The model performs well on the validation set and this configuration is chosen as the final model.

Process the test set data.

In [0]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

Evaluate the model on the test set.

In [60]:
accr = model.evaluate(test_sequences_matrix,Y_test)

5399/5399 [==============================] - 47s 9ms/step


In [61]:
accr

[0.16680728948145007, 0.9383219115092459]

In [0]:
y_pred = model.predict(test_sequences_matrix)

In [0]:
for i in range(len(y_pred)):
    if y_pred[i] < 0.4:
        y_pred[i] = 0
    else:
        y_pred[i] = 1

In [64]:
from sklearn.metrics import precision_score,recall_score
print ('F1 score:', f1_score(Y_test, y_pred,average='macro'))
print ('Recall:', recall_score(Y_test, y_pred,average='macro'))
print ('Precision:', precision_score(Y_test, y_pred,average='macro'))

F1 score: 0.8449361355624079
Recall: 0.8599708709545095
Precision: 0.831610708659889


In [65]:
from sklearn.metrics import f1_score, fbeta_score
f1_score(Y_test, y_pred, average='macro')

0.8665808513002511

In [0]:

f = io.open("task1.test.txt", mode="r", encoding="utf-8")
df = pd.DataFrame(columns=["text","article_id","propaganda"])
lines = f.read().split("\n")
x = []
for i in lines:
    i = i.split("\t")
    x.append(i)
df = pd.DataFrame(x, columns=["text","article_id","propaganda"])
df.head()

,text,article_id,propaganda
0,Chicago police are seeking the public’s help i...,100013,?
1,Moment of silence at Palestine Pavilion during...,100015,?
2,The Organization of Islamic Cooperation (OIC) ...,100024,?
3,The chairman of the House Oversight and Govern...,100031,?
4,A judge in the United Kingdom has sentenced a ...,100040,?


In [0]:
preprocessed_text = []
# tqdm is for printing the status bar
for sentance in tqdm(df['text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    #sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_text.append(sentance.strip())

100%|██████████| 10159/10159 [00:13<00:00, 736.32it/s]


In [0]:
df["text"] = preprocessed_text
df.head()
df = df.dropna()

In [0]:
X = df.text

In [0]:
article_id = df.article_id

In [0]:
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [0]:
y_pred = model.predict(sequences_matrix)

In [0]:
for i in range(len(y_pred)):
    if y_pred[i] < 0.4:
        y_pred[i] = 0
    else:
        y_pred[i] = 1

In [0]:
preds = []
for i in range(len(y_pred)):
  preds.append(int(y_pred[i][0]))

In [0]:
df_submission = pd.DataFrame(
    {'article_id': article_id,
     'propaganda/non-propaganda': preds
    })